In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nanomesh.volume import Volume
from nanomesh.plane import Plane

In [ ]:
vol = Volume.load('sample_data.npy')

In [ ]:
vol.show_slice()

In [ ]:
plane = vol.select_plane(x=161)

In [ ]:
plane.show()

In [ ]:
from skimage import filters

In [ ]:
plane_gauss = plane.apply_np(filters.gaussian, sigma=5)
plane_gauss.show()

In [ ]:
filters.try_all_threshold(plane_gauss.array_view, figsize=(10, 15), verbose=True)

In [ ]:
thresh = filters.threshold_li(plane_gauss.array_view)

seg = Plane.from_array(1.0 * (plane_gauss.array_view >= thresh))
seg.show()

In [ ]:
mesh = seg.generate_mesh(pad=True, plot=True)

In [ ]:
import pyvista as pv
pv.plot_itk(mesh)

In [ ]:
mesh.write("plane.msh")

In [ ]:
import ipyvolume as ipv

ipv.figure()
# we draw the tetrahedron
ipv_mesh = ipv.plot_trisurf(*mesh.points.T, triangles=mesh.cells[0].data, color='orange')
# and also mark the vertices
ipv.scatter(*mesh.points.T, marker='sphere', color='blue')
ipv.xyzlim(-2, 2)
ipv.show()

## Using pygalmesh

In [ ]:
from matplotlib.tri.triplot import Triangulation

x, y, _ = mesh.points.T
triangles = mesh.cells[0].data
tri, _, _ = Triangulation.get_from_args_and_kwargs(x, y, triangles=triangles)

import pygalmesh

points = np.stack((tri.x, tri.y)).T

mesh = pygalmesh.generate_2d(
    points=points,
    constraints=tri.edges,
    max_edge_size=50,
    num_lloyd_steps=10,
)
# mesh.points, mesh.cells

pv.plot_itk(mesh)